In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import pandas as pd
from pandas import DataFrame
import lxml
! pip install folium
! pip install geopy
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
import requests
import types
from botocore.client import Config
import ibm_boto3


In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df_data_1 = pd.read_csv(body) #import cleveland neighborhood data from source file
df_data_1.head()

,OBJECTID,MUNI_ID,MUNI_NAME,CENSUS_ID,CALC_AREA,SURVEY_ACREAGE,SHAPE_Length,SHAPE_Area
0,1,1,Mayfield,48468.0,3.936042,NaN,21821.722876,1.821751e+07
1,2,2,Lakewood,41664.0,5.793805,NaN,29324.756515,2.674328e+07
2,3,3,Garfield Heights,29428.0,7.238719,NaN,31168.752868,3.337193e+07
3,4,4,Moreland Hills,52052.0,7.257437,NaN,30491.423324,3.348269e+07
4,5,5,Middleburg Heights,NaN,8.061551,NaN,29195.563224,3.711125e+07


In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
df_data_2 = pd.read_csv(body) # import latitude and longitude data from source file
df_data_2.head()

,zip_code,latitude,longitude,city,state,county
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk
2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,602,18.393103,-67.180953,Aguada,PR,Aguada
4,603,18.455913,-67.145780,Aguadilla,PR,Aguadilla


In [6]:
# create dataframe with names of cities with corresponding latitude and longitude values for cities in Cleveland 
df_ohio=df_data_2.loc[df_data_2['county'] == "Cuyahoga"]
df_ohio.dropna(inplace=True)
df_ohio.drop(df_ohio[df_ohio["latitude"]==41.685744].index, inplace = True)
df_ohio.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,zip_code,latitude,longitude,city,state,county
19028,44017,41.456905,-81.693019,Berea,OH,Cuyahoga
19030,44022,41.449353,-81.431436,Chagrin Falls,OH,Cuyahoga
19041,44040,41.513251,-81.414597,Gates Mills,OH,Cuyahoga
19063,44070,41.417347,-81.922461,North Olmsted,OH,Cuyahoga
19086,44101,41.523401,-81.599648,Cleveland,OH,Cuyahoga


In [7]:
#get location data about Cleveland

address = 'Cleveland, OH'

geolocator = Nominatim(user_agent="cleveland neighborhoods")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Cleveland using latitude and longitude values
map_cleveland_cities = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to mark the cities in Cleveland
for lat, lng, label in zip(df_ohio['latitude'], df_ohio['longitude'], df_ohio['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cleveland_cities)  
    
map_cleveland_cities

In [8]:
#clean data to use easily for kMeans clustering
df_ohio_clustering=df_ohio.drop(["zip_code", "city","state", "county"], axis=1)

In [9]:
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_ohio_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add cluster labels to data about Cleveland cities
df_ohio.insert(0, 'Cluster Labels', kmeans.labels_)
df_ohio.head()

,Cluster Labels,zip_code,latitude,longitude,city,state,county
19028,2,44017,41.456905,-81.693019,Berea,OH,Cuyahoga
19030,1,44022,41.449353,-81.431436,Chagrin Falls,OH,Cuyahoga
19041,1,44040,41.513251,-81.414597,Gates Mills,OH,Cuyahoga
19063,0,44070,41.417347,-81.922461,North Olmsted,OH,Cuyahoga
19086,2,44101,41.523401,-81.599648,Cleveland,OH,Cuyahoga


In [10]:
# create map that shows 3 neighborhood clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_ohio['latitude'], df_ohio['longitude'], df_ohio['city'], df_ohio['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [11]:
#determine the location of the centroid of each cluster and save info in new dataframe
centroids  = kmeans.cluster_centers_
df_centroids=pd.DataFrame({"Cluster Label": [0,1,2],"latitude": centroids[:,0], "longitude": centroids[:,1]})
df_centroids.head()

,Cluster Label,latitude,longitude
0,0,41.421065,-81.816800
1,1,41.520168,-81.505749
2,2,41.458201,-81.633398


In [12]:
# The code was removed by Watson Studio for sharing.

# Western Cleveland Neighborhoods


In [13]:
#call the foursquare API to determine the first 100 restaurants within 8000 meters (about 5 miles) of the centriod of the Western Region
#set location to centriod of western Neighborhoods
latitude = df_centroids.loc[0,"latitude"]
longitude = df_centroids.loc[0,"longitude"]

category_Id = "4d4b7105d754a06374d81259"
radius = 8000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_Id, radius, LIMIT)
results = requests.get(url).json()

In [14]:
# get the wanted data from the results of the FourSquare call
venues = results['response']['venues']

# tranform venues into a dataframe and clean the data about the restaurants
dataframe = json_normalize(venues)
food_west = dataframe[["name","id","location.lat","location.lng"]]
food_west.set_index('name', inplace=True)
food_west.head()

,id,location.lat,location.lng
name,,,
Express Deli & Beverage,4b703836f964a5203d0c2de3,41.413594,-81.801297
Fat Head's Brewery & Saloon,4ae39242f964a520ca9621e3,41.426314,-81.894213
Raising Cane's Chicken Fingers,5d8ac776dcfa830008de60c3,41.485283,-81.790991
Starbucks,4aeb75cdf964a5204dc221e3,41.462602,-81.849488
Dunkin',4bbf1547b083a5936ccfa2e9,41.425852,-81.734937


In [15]:
#Call the FourSquare API to determine the user rating for each of the restaurants in the dataframe

id_list = food_west['id'].tolist()
food_west_ratings = []
for i in id_list:
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    food_west_venueinfo = json_normalize(result)
    try:
        rating=food_west_venueinfo["response.venue.rating"].tolist()
        food_west_ratings.append(rating)
    except:
        food_west_ratings.append("X")
        
from itertools import chain
food_west_ratings = list(chain(*food_west_ratings))

food_west['rating']= food_west_ratings
food_west.head()
                                          

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,location.lat,location.lng,rating
name,,,,
Express Deli & Beverage,4b703836f964a5203d0c2de3,41.413594,-81.801297,8.5
Fat Head's Brewery & Saloon,4ae39242f964a520ca9621e3,41.426314,-81.894213,9.1
Raising Cane's Chicken Fingers,5d8ac776dcfa830008de60c3,41.485283,-81.790991,7.9
Starbucks,4aeb75cdf964a5204dc221e3,41.462602,-81.849488,7.5
Dunkin',4bbf1547b083a5936ccfa2e9,41.425852,-81.734937,7.5


In [16]:
# Drop the restuarants without a rating so that the average rating can be calculated
food_west.drop(food_west.loc[food_west['rating']=="X"].index, inplace=True)
west_mean = food_west['rating'].mean()
print("the average restaurant rating in the western neighborhoods is") 
print(west_mean)


the average restaurant rating in the western neighborhoods is
7.884090909090907


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
# Just for fun, determine the highest rated restaurant in the neighborhood
max = food_west['rating'].max()
west_max = food_west.loc[food_west['rating'] == max]
print("the best rated restaurant in the western neighborhoods is") 
print(west_max.index.values)

the best rated restaurant in the western neighborhoods is
["Angelo's Pizza"]


# Eastern Cleveland Neighborhoods

In [18]:
#repeat steps for eastern neighborhoods

latitude = df_centroids.loc[1,"latitude"]
longitude = df_centroids.loc[1,"longitude"]

In [ ]:
category_Id = "4d4b7105d754a06374d81259"
radius = 8000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_Id, radius, LIMIT)
results = requests.get(url).json()

In [ ]:
venues = results['response']['venues']
dataframe = json_normalize(venues) # tranform venues into a dataframe
food_east = dataframe[["name","id","location.lat","location.lng"]]
food_east.set_index('name', inplace=True)

In [ ]:
id_list = food_east['id'].tolist()
food_east_ratings = []
for i in id_list:
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    food_east_venueinfo = json_normalize(result)
    try:
        rating=food_east_venueinfo["response.venue.rating"].tolist()
        food_east_ratings.append(rating)
    except:
        food_east_ratings.append("X")
        
from itertools import chain
food_east_ratings = list(chain(*food_east_ratings))

food_east['rating']= food_east_ratings
food_east.head()
                  

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,location.lat,location.lng,rating
name,,,,
The Mayfield Sand Ridge Club,4c325f93ed37a59317c56b03,41.515582,-81.509689,7.9
Condado Tacos,5bca07a786bc490039060bbe,41.454693,-81.486652,8
Taco Bell,5b6a500cb9a5a8002c541506,41.520390,-81.468730,6.3
Wendy’s,4b380423f964a520504a25e3,41.538053,-81.497678,5.8
Sunday Memories,57cf6084498e312d85afccd9,41.539574,-81.478090,X


In [ ]:
food_east.drop(food_east.loc[food_east['rating']=="X"].index, inplace=True)
east_mean = food_east['rating'].mean()
print("the average restaurant rating in the eastern neighborhoods is") 
print(east_mean)

the average restaurant rating in the eastern neighborhoods is
7.428260869565216


In [ ]:
max = food_east['rating'].max()
east_max = food_east.loc[food_east['rating'] == max]
print("the best rated restaurant in the eastern neighborhoods is") 
print(east_max.index.values)

the best rated restaurant in the eastern neighborhoods is
["Presti's Bakery"]


 # Central Cleveland Neighborhoods

In [ ]:
# Repeat steps for Central Neighborhoods
latitude = df_centroids.loc[2,"latitude"]
longitude = df_centroids.loc[2,"longitude"]

In [ ]:
category_Id = "4d4b7105d754a06374d81259"
radius = 8000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, category_Id, radius, LIMIT)
results = requests.get(url).json()

In [ ]:
venues = results['response']['venues']
dataframe = json_normalize(venues) # tranform venues into a dataframe
food_central = dataframe[["name","id","location.lat","location.lng"]]
food_central.set_index('name', inplace=True)
food_central.head()

,id,location.lat,location.lng
name,,,
McDonald's,4bbdc648a8cf76b0a61db2fd,41.472745,-81.622614
Lago Cucina Enoteca and Birreria,51d4d21d498e518d61e891fc,41.500606,-81.704032
Lindey’s Lake House,5ca93e00a6031c002ca971d7,41.499164,-81.704980
Dunkin',4bbf1547b083a5936ccfa2e9,41.425852,-81.734937
Titans Gym,4be5ab7b2468c92896580043,41.498777,-81.688768


In [ ]:
id_list = food_central['id'].tolist()
food_central_ratings = []
for i in id_list:
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    food_central_venueinfo = json_normalize(result)
    try:
        rating=food_central_venueinfo["response.venue.rating"].tolist()
        food_central_ratings.append(rating)
    except:
        food_central_ratings.append("X")
        
from itertools import chain
food_central_ratings = list(chain(*food_central_ratings))

food_central['rating']= food_central_ratings
food_central.head()
                

In [ ]:
food_central.drop(food_central.loc[food_central['rating']=="X"].index, inplace=True)
central_mean = food_central['rating'].mean()
print("the average restaurant rating in the central neighborhoods is") 
print(central_mean)

In [ ]:
max = food_central['rating'].max()
central_max = food_central.loc[food_central['rating'] == max]
print("the best rated restaurant in the central neighborhoods is") 
print(central_max.index.values)

# Final Comparision

In [ ]:
east_mean, west_mean, central_mean